In [0]:
dbutils.widgets.text("workspace_url", "")
dbutils.widgets.text("sp_secret_scope", "")
dbutils.widgets.text("sp_secret_key_client_id", "")
dbutils.widgets.text("sp_secret_key_secret", "")

In [0]:
workspace_url = dbutils.widgets.get("workspace_url")
sp_secret_scope = dbutils.widgets.get("sp_secret_scope")
sp_secret_key_client_id = dbutils.widgets.get("sp_secret_key_client_id")
sp_secret_key_secret = dbutils.widgets.get("sp_secret_key_secret")

In [0]:
print(f"""
workspace_url: {workspace_url}
sp_secret_scope: {sp_secret_scope}
sp_secret_key_client_id: {sp_secret_key_client_id}
sp_secret_key_secret: {sp_secret_key_secret}      
""")

In [0]:
service_principal_client_id = dbutils.secrets.get(sp_secret_scope, sp_secret_key_client_id)
service_principal_client_secret = dbutils.secrets.get(sp_secret_scope, sp_secret_key_secret)

In [0]:
print(f"""
service_principal_client_id: {service_principal_client_id}
service_principal_client_secret: {service_principal_client_secret}
""")

In [0]:
# Requires: pip install databricks-vectorsearch
import time
from databricks.vector_search.client import VectorSearchClient

In [0]:
def ensure_vector_search_endpoint(
	name: str
	,endpoint_type: str = "STANDARD"           # or "STORAGE_OPTIMIZED"
	,timeout_secs: int = 20 * 60               # wait up to 20 minutes
	,poll_interval_secs: int = 10
	,budget_policy_id: str | None = None       # optional budget policy to attach
	,client: VectorSearchClient | None = None  # pass your own client if desired
) -> dict:
	"""
	Idempotently ensure a Vector Search endpoint exists and is ONLINE.
	Returns the endpoint dict from get_endpoint().
	"""
	vsc = client or VectorSearchClient()  # auto-auth in notebooks; or pass SP creds

	# Check if endpoint exists
	endpoints = vsc.list_endpoints().get("endpoints", [])
	if not any(ep.get("name") == name for ep in endpoints):
		vsc.create_endpoint(
			name=name
			,endpoint_type=endpoint_type
		)

	# Optional: apply/update a budget policy
	if budget_policy_id:
		try:
			vsc.update_endpoint_budget_policy(
				name=name
				,budget_policy_id=budget_policy_id
			)
		except Exception as e:
			# non-fatal; you can choose to raise instead
			print(f"Warning: couldn’t set budget policy: {e}")

	# Wait until ONLINE (or timeout)
	deadline = time.time() + timeout_secs
	last_state = None
	while time.time() < deadline:
		ep = vsc.get_endpoint(name=name)
		state = ep.get("endpoint_status", {}).get("state")
		if state != last_state:
			print(f"Endpoint '{name}' state: {state}")
			last_state = state
		if state == "ONLINE":
			return ep
		time.sleep(poll_interval_secs)

	raise TimeoutError(f"Endpoint '{name}' did not become ONLINE within {timeout_secs}s")

In [0]:
vs_client = VectorSearchClient(
	workspace_url=workspace_url
	,service_principal_client_id=service_principal_client_id
	,service_principal_client_secret=service_principal_client_secret
)

In [0]:
# vs_client.list_endpoints()

In [0]:
# vs_client.get_endpoint("ncpdp-specifications-vs-endpoint")

In [0]:
ensure_vector_search_endpoint(
    "ncpdp-specifications-vs-endpoint"
    ,client=vs_client
)